In [1]:
import psycopg2
import pandas as pd

# Database connection details
username = 'test'
password = 'test123'
hostname = '192.168.1.141'
port = '5432'
database_name = 'mydb'

# Establish connection
conn = psycopg2.connect(
    dbname=database_name,
    user=username,
    password=password,
    host=hostname,
    port=port
)

# SQL query
sql_query = "SELECT  TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS lasttradetime, open,high,low,close,openinterest FROM whole_indices_futures WHERE instrumentidentifier = 'NIFTY10APR2422700CE' order by lasttradetime;"

# Execute SQL query
cursor = conn.cursor()
cursor.execute(sql_query)

# Fetch all the results
rows = cursor.fetchall()

# Convert to DataFrame
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close cursor and connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


            lasttradetime    open    high     low   close  openinterest
0     2024-03-13 10:07:00  460.00  460.00  460.00  460.00             0
1     2024-03-13 10:07:00  460.00  460.00  460.00  460.00             0
2     2024-03-13 10:08:00  460.00  460.00  460.00  460.00             0
3     2024-03-13 10:08:00  460.00  460.00  460.00  460.00             0
4     2024-03-13 10:09:00  460.00  460.00  460.00  460.00             0
...                   ...     ...     ...     ...     ...           ...
11165 2024-04-09 15:28:00  249.35  249.60  241.20  244.45        916110
11166 2024-04-09 15:29:00  244.45  249.45  244.45  249.10        916110
11167 2024-04-09 15:29:00  244.45  249.45  244.45  249.10        916110
11168 2024-04-09 15:30:00  245.30  245.30  245.30  245.30        916110
11169 2024-04-09 15:30:00  245.30  245.30  245.30  245.30        916110

[11170 rows x 6 columns]


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

df.set_index('lasttradetime', inplace=True)
df.fillna(method="ffill", inplace=True)	
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
lookback_window = 3
features = 5
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:5]) 
    return np.array(X), np.array(y)
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=100))
model.add(Dense(units=5)) 
model.compile(loss='mean_squared_error', optimizer='adam')
X, y = create_sequences(df_scaled, lookback_window)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','tradedqty'])
data


C:\Users\santh\AppData\Local\Temp\ipykernel_15052\3742571262.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0074 - val_loss: 4.4091e-04
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.1326e-04 - val_loss: 3.8926e-04
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 9.0082e-05 - val_loss: 5.5249e-04
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.0756e-05 - val_loss: 3.6939e-04
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.3816e-04 - val_loss: 4.2996e-04
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 8.2094e-05 - val_loss: 4.7723e-04
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 7.6299e-05 - val_loss: 4.5165e-04
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.5152e-05 - val_loss: 6.0496e-04
Epoch 9/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.9546e-04 - val_loss: 4.9755e-04
Epoch 10/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.7240e-05 - val_loss: 3.9839e-04
Epoch 11/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.

,open,high,low,close,tradedqty
0,134.724182,140.765381,131.925568,136.269684,894506.1875
1,135.200592,141.042557,131.755920,136.010483,891657.1875
2,135.734085,141.722504,132.013992,136.469604,892320.2500
3,137.082779,142.754822,133.547989,137.769775,892439.8750
4,137.244370,141.994202,133.493958,137.080917,890806.5625
...,...,...,...,...,...
2229,247.238983,256.859344,240.593109,247.706543,955819.0625
2230,246.071014,255.261734,238.917038,245.890625,932163.1250
2231,245.495880,254.703156,238.258347,245.282288,902124.4375
2232,246.107086,255.679337,239.434998,246.582443,871058.6875


In [3]:
def generate_predictions(model, last_sequence, num_predictions):
    predictions = []
    current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
    for _ in range(num_predictions):
        next_prediction = model.predict(current_sequence)  # Predict the next data point
        predictions.append(next_prediction[0])  # Append the prediction to the list of predictions
        current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
    return predictions

# Generate predictions for the next 60 data points
last_sequence = X_test[-1]  # Using the last sequence from the test data
num_predictions = 720
predicted_data = generate_predictions(model, last_sequence, num_predictions)

# Inverse transform the predictions to obtain the original scale
predicted_data_original = scaler.inverse_transform(predicted_data)

# Create a DataFrame with the predicted values and column names
predicted_df = pd.DataFrame(predicted_data_original, columns=['open', 'high', 'low', 'close', 'volume'])

# Print the predicted data
print(predicted_df)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

In [4]:
frequency = 'T'
start_time = '2024-04-09 15:30:00'
end_time = pd.to_datetime(start_time) + pd.DateOffset(minutes=len(predicted_df) - 1)
date_range = pd.date_range(start=start_time, end=end_time, freq=frequency)
predicted_df['lasttradetime'] = date_range
predicted_df.set_index('lasttradetime', inplace=True)
predicted_df.to_csv('BANKNIFTY10APR2448600CE.csv')


predicted_df

C:\Users\santh\AppData\Local\Temp\ipykernel_15052\3403744794.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=start_time, end=end_time, freq=frequency)


,open,high,low,close,volume
lasttradetime,,,,,
2024-04-09 15:30:00,246.137190,255.602020,239.560605,246.590154,870602.599525
2024-04-09 15:31:00,246.666875,256.564974,239.817866,247.160508,859830.891645
2024-04-09 15:32:00,246.968110,257.372227,239.699493,247.383087,841717.053115
2024-04-09 15:33:00,247.145100,258.012932,239.595615,247.529774,817789.035416
2024-04-09 15:34:00,247.555440,258.418083,239.965266,247.903314,801370.995498
...,...,...,...,...,...
2024-04-10 03:25:00,445.944480,443.220990,450.104505,446.264460,-121708.183077
2024-04-10 03:26:00,445.944480,443.221016,450.104505,446.264485,-121708.161199
2024-04-10 03:27:00,445.944505,443.220990,450.104505,446.264485,-121708.095565
